In [6]:
import requests
import numpy as np 
import scipy
from bs4 import BeautifulSoup

In [7]:
page = requests.get('https://www.mcgill.ca/study/2021-2022/faculties/science/undergraduate/programs/bachelor-science-bsc-major-computer-science-and-biology')
soup = BeautifulSoup(page.content, 'html.parser')
title = soup.title.text # gets you the text of the <title>(...)</title>
print(title)

Bachelor of Science (B.Sc.) - Major Computer Science and Biology | eCalendar - McGill University


In [98]:
major_courses = soup.find_all(class_="program-course-title")
major_courses = [major_courses[i].contents[0][15:major_courses[i].contents[0].index(")")+1] for i in range(0, len(major_courses))]
major_courses

['BIOL 219 Introduction to Physical\r Molecular and Cell Biology (4 credits)',
 'BIOL 301 Cell and Molecular Laboratory (4 credits)',
 'BIOL 395 Quantitative Biology Seminar (1 credit)',
 'CHEM 212 Introductory Organic Chemistry 1 (4 credits)',
 'COMP 202 Foundations of Programming (3 credits)',
 'MATH 222 Calculus 3 (3 credits)',
 'MATH 223 Linear Algebra (3 credits)',
 'MATH 323 Probability (3 credits)',
 'COMP 204 Computer Programming for Life Sciences (3 credits)',
 'COMP 206 Introduction to Software Systems (3 credits)',
 'COMP 250 Introduction to Computer Science (3 credits)',
 'COMP 251 Algorithms and Data Structures (3 credits)',
 'MATH 240 Discrete Structures (3 credits)',
 'BIOL 202 Basic Genetics (3 credits)',
 'BIOL 215 Introduction to Ecology and Evolution (3 credits)',
 'COMP 401 Project in Biology and Computer Science (3 credits)',
 'COMP 462 Computational Biology Methods (3 credits)',
 'COMP 561 Computational Biology Methods and Research (4 credits)',
 'MATH 315 Ordinar